In [1]:
import pandas as pd
import importlib
from datetime import date
import matplotlib.pyplot as plt

In [2]:
import modo_energy_client.modo_energy_api_client

importlib.reload(modo_energy_client.modo_energy_api_client)
from modo_energy_client.modo_energy_api_client import ModoEnergyAPIClient

client = ModoEnergyAPIClient(api_token="your_api_token_here")

In [3]:
df = client.get_ercot_modo_owners(date_from="2010-01", date_to="2026-01")

Fetching pages : 0page [00:00, ?page/s]

In [5]:
df.dtypes

date                         datetime64[us]
total_energy_capacity_mwh           float64
total_rated_power_mw                float64
asset_count                           int64
owner                                   str
dtype: object

In [ ]:
# Example: adjust these column names as needed based on the printout
df_grouped = (
    df.groupby("date")
    .agg({"total_energy_capacity_mwh": "sum", "total_rated_power_mw": "sum"})
    .reset_index()
)

plt.figure(figsize=(12, 6))
plt.plot(
    df_grouped["date"],
    df_grouped["total_energy_capacity_mwh"],
    label="Total Energy Capacity (MWh)",
)
plt.plot(
    df_grouped["date"], df_grouped["total_rated_power_mw"], label="Rated Power (MW)"
)
plt.xlabel("Date")
plt.ylabel("Capacity / Power")
plt.title("Change in Total BESS Energy Capacity and Rated Power Over Time")
plt.legend()

# Show only one date label per quarter
dates = pd.to_datetime(df_grouped["date"])
quarter_ticks = [
    i for i, d in enumerate(dates) if d.month in [1, 4, 7, 10] and d.day == 1
]
plt.xticks(quarter_ticks, [df_grouped["date"][i] for i in quarter_ticks], rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
fuel_mix = client.get_ercot_generation_fuel_mix(
    date_from=date(2024, 1, 1), date_to=date(2025, 12, 31)
)

In [ ]:
fuel_mix.resample("ME").sum().plot()